In [5]:
import os, shutil
from keras import layers
from keras import models
from keras import optimizers
import math

Using TensorFlow backend.


In [2]:
#List of categories that will be considered. The items of this list have the same name as the names of the folders 
#the images are strored in.

list_categories = ['bar','bowling','buffet','casino','concert_hall','fastfood_restaurant','gameroom','gym',
                  'hairsalon','movietheater','restaurant','airport_inside','church_inside','cloister','elevator',
                  'florist','inside_bus','library','locker_room','museum','poolinside','prisoncell','subway',
                   'trainstation','waitingroom']

In [6]:
# Next we create the file structure to split the images in train, validation and test sets.


#Where the original images are stored.
original_dataset_dir = 'original_images'

#Where we will store the images separated by train, validation and test sets.
base_dir = 'images'

os.mkdir(base_dir)

train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir,'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

In [9]:
#Inside each of the folders created peveously, we create a folder for each of the categories.

or i in range(len(list_categories)):
    os.mkdir(os.path.join(train_dir,list_categories[i]))
    
for i in range(len(list_categories)):
    os.mkdir(os.path.join(validation_dir,list_categories[i]))
    
for i in range(len(list_categories)):
    os.mkdir(os.path.join(test_dir,list_categories[i]))

In [24]:
#Next we divide the images into train(60%), validation(20%) and test(20%) and copy them to the new file structure.

for i in range(len(list_categories)):
    directory = os.path.join(original_dataset_dir,list_categories[i])
    number_of_files = sum(1 for item in os.listdir(directory) if os.path.isfile(os.path.join(directory, item)))
    train_indice = math.ceil(number_of_files*0.6)
    validation_indice = math.ceil(number_of_files*0.8)
    iterator = 0
    for file in os.listdir(directory):
        if iterator < train_indice:
            src = os.path.join(directory,file)
            dst = os.path.join(train_dir,list_categories[i],file)
            shutil.copyfile(src,dst)
        elif iterator < validation_indice:
            src = os.path.join(directory,file)
            dst = os.path.join(validation_dir,list_categories[i],file)
            shutil.copyfile(src,dst)
        else:
            src = os.path.join(directory,file)
            dst = os.path.join(test_dir,list_categories[i],file)
            shutil.copyfile(src,dst)
        iterator += 1

In [34]:
#Model definition

model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
          
model.add(layers.Flatten())
          
model.add(layers.Dense(512,activation='relu'))

model.add(layers.Dense(25,activation='sigmoid'))

In [35]:
#Model Compilation

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [36]:
#Data genaration for the training and validation steps

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150,150),
        batch_size=20,
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(150,150),
        batch_size=20,
        class_mode='categorical')

Found 3492 images belonging to 25 classes.
Found 1163 images belonging to 25 classes.


In [37]:
for data_batch, labels_batch in train_generator:
    print('data batch shape: ', data_batch.shape)
    print('labels batch shape: ', labels_batch.shape)
    break

data batch shape:  (20, 150, 150, 3)
labels batch shape:  (20, 25)


In [38]:
#Fitting the model:

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/30
100/100 [==============================] - 40s 400ms/step - loss: 3.0072 - acc: 0.1029 - val_loss: 2.9989 - val_acc: 0.1060
Epoch 2/30
100/100 [==============================] - 47s 466ms/step - loss: 2.8663 - acc: 0.1095 - val_loss: 3.4432 - val_acc: 0.1170
Epoch 3/30
100/100 [==============================] - 44s 444ms/step - loss: 2.7152 - acc: 0.1606 - val_loss: 2.8297 - val_acc: 0.1913
Epoch 4/30
100/100 [==============================] - 46s 460ms/step - loss: 2.6374 - acc: 0.2025 - val_loss: 2.9245 - val_acc: 0.2350
Epoch 5/30
100/100 [==============================] - 46s 459ms/step - loss: 2.4979 - acc: 0.2324 - val_loss: 2.5471 - val_acc: 0.2055
Epoch 6/30
100/100 [==============================] - 47s 465ms/step - loss: 2.4455 - acc: 0.2580 - val_loss: 2.6428 - val_acc: 0.2289
Epoch 7/30
100/100 [==============================] - 51s 510ms/step - loss: 2.3873 - acc: 0.2725 - val_loss: 2.2681 - val_acc: 0.2520
Epoch 8/30
100/100 [==============================] - 5

In [78]:
#Data generation for the test step:

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (150, 150),
                                            batch_size = 20,
                                            class_mode = 'categorical')

Found 1150 images belonging to 25 classes.


In [83]:
#Evaluate model

model.evaluate_generator(test_set)

[1.796738862991333, 0.3199999928474426]